In [ ]:
 import zipfile
from PIL import Image
import numpy as np
import io
import tensorly as tly
from tensorly.tenalg import mode_dot
from scipy.linalg import lstsq

In the first box, I created matrices A for each person.

In [ ]:
zip_file = 'archive.zip'

desired_size = (192,168)

image_data_dict = {}
all_images = [] 

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file_name in file_list:
            subject_id = file_name.split('/')[1].split('_')[0]
            image_data = zip_ref.read(file_name)
            img = Image.open(io.BytesIO(image_data))
            img_resized = img.resize(desired_size)
    
            img_array = np.array(img_resized).flatten()
            all_images.append(img_array)

            if subject_id in image_data_dict:
                image_data_dict[subject_id]['FlattenedVectors'].append(img_array)
            else:
                image_data_dict[subject_id] = {
                    'FlattenedVectors': [img_array],
                }
                
                
matrices_list=[]
for subject_id, data in image_data_dict.items():
    concatenated_matrix = np.column_stack(data['FlattenedVectors'])
    matrices_list.append(concatenated_matrix)
    print(f"matrix for {subject_id}:\n{concatenated_matrix}\n")


In the next boxes, I created the desired tensor.

In [ ]:
resize = (32256,65)
resized_matrices = [np.resize(matrix, resize) for matrix in matrices_list]

tensor = np.stack(resized_matrices, axis=2)

print("3-Mode Tensor Shape:", tensor.shape)
print(tensor)

Now, we have the tensor and it's time to apply HOSVD.

In [ ]:
dim1, dim2, dim3 = tensor.shape

matrix_1 = tensor.reshape(dim1, -1)
F, _, _ = np.linalg.svd(matrix_1, full_matrices=False)

matrix_2 = tensor.transpose(1, 0, 2).reshape(dim2, -1)
G, _, _ = np.linalg.svd(matrix_2, full_matrices=False)

matrix_3 = tensor.transpose(2, 0, 1).reshape(dim3, -1)
H, _, _ = np.linalg.svd(matrix_3, full_matrices=False)


In [ ]:
print("\nF:")
print(F)
print("\nG:")
print(G)
print("\nH:")
print(H)

In [ ]:
 S = mode_dot(mode_dot(mode_dot(tensor,F.T,0), G.T,1), H.T,2)

In [ ]:
print("\nS:")
print(S)

Now, It,s time to do the first approach.
First, we create C which is created by multiplying S and F and G and the tensor2 which is created by multiplying C and H.

In [ ]:
C = tly.tenalg.mode_dot(S, F, mode=0)
C = tly.tenalg.mode_dot(C, G, mode=1)
tensor2 = tly.tenalg.mode_dot(C, H, mode=2)

for each expression e, we need to minimize ‖𝐶𝑒𝛼𝑒 − 𝑧‖ and z will be the image of the person with the least amount of this norm.

In [ ]:
#z is the given image that we want to classify.
z = all_images[2]
aes = []
norms = []

for i in range(0,65):
    ce = C[ : , i , :]
    ae, _, _, _ = lstsq(ce, z)
    norm = np.linalg.norm(np.dot(ce,ae) - z) 
    norms.append(norm)
    aes.append(ae)


In [ ]:
res = min(norms)
res

In [ ]:
norms.index(res)